In [1]:
# This works. Processes videos in np arrays, which we can save to folder or use directly

import cv2, os
print(cv2.__version__)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
import pandas as pd 
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

from IPython.core.display import Image, display

import numpy as np
import shutil

3.4.2


In [2]:
!ls '../../videos/'

2018_09_08_15_08_54	 DJI_0020_lr.MP4
2018_09_08_15_08_54.mp4  DJI_0021_lr
9.27_lazy_overhead	 DJI_0021_lr.MP4
DJI_0006.MP4		 DJI_0023_lr.MP4
DJI_0007_lr		 DJI_0024_lr.MP4
DJI_0007_lr.MP4		 DJI_0025_lr.MP4
DJI_0018_lr		 DJIFlightRecord_2018-09-18_[18-15-41]-TxtLogToCsv.csv
DJI_0018_lr.MP4		 DJIFlightRecord_2018-09-18_[18-27-45]-TxtLogToCsv.csv
DJI_0019_lr		 DJIFlightRecord_2018-09-18_[18-36-05]-TxtLogToCsv.csv
DJI_0019_lr.MP4		 DJIFlightRecord_2018-09-18_[18-42-50]-TxtLogToCsv.csv
DJI_0020_lr		 videos


In [32]:
! rm ../../videos/DJI_0022.MP4

In [30]:
shutil.rmtree('../../videos/DJI_0002')

FileNotFoundError: [Errno 2] No such file or directory: '../../videos/DJI_0002'

In [3]:
def format_ix(ix): # Takes in int and returns str padded w 0s. Want to keep files in order. Good up to 99k.
    return ('0000'+str(ix))[-5:]

## Folder structure

-videos
    -group_folder 
        vid1.mp4
        vid2.mp4
        vid3.mp4
        actions1.csv
        actions2.csv
        actions3.csv
        -frames
           -vid1
               frame_1
               frame_2
           -vid2
               frame_1
               frame_2
           -vid3
               frame_1
               frame_2
               frame_3
               
Dataset will be given the videos/group_folder/frames directory, which contains the frames of all vids

In [4]:
group_folder = "9.27_lazy_overhead" # this is main folder holds videos, csvs for group

In [8]:
# Move vidfolder from bucket to instance

os.system('sudo gsutil cp -r gs://bean-warehouse/'+group_folder+' ../../videos')

0

In [34]:
# Getting folder structure in order

group_folder_path = "../../videos/"+group_folder
frames_path = group_folder_path +"/frames/"
os.system('sudo mkdir '+frames_path)
vid_ids = ["DJI_0040_lr", "DJI_0041_lr", "DJI_0042_lr", "DJI_0043_lr"] # first one didn't get called lr

In [35]:
group_folder_path

'../../videos/9.27_lazy_overhead'

In [16]:
!ls '../../videos/9.27_lazy_overhead'

DJI_0026_lr.MP4  DJI_0040_lr.MP4
DJI_0027_lr.MP4  DJI_0041_lr.MP4
DJI_0028_lr.MP4  DJI_0042_lr.MP4
DJI_0029_lr.MP4  DJI_0043_lr.MP4
DJI_0030_lr.MP4  DJIFlightRecord_2018-09-27_[16-20-07]-TxtLogToCsv.csv
DJI_0031_lr.MP4  DJIFlightRecord_2018-09-30_[08-28-21]-TxtLogToCsv.csv
DJI_0033_lr.MP4  DJIFlightRecord_2018-09-30_[08-35-09]-TxtLogToCsv.csv
DJI_0034_lr.MP4  DJIFlightRecord_2018-09-30_[09-58-56]-TxtLogToCsv.csv
DJI_0035_lr.MP4  DJIFlightRecord_2018-09-30_[10-12-16]-TxtLogToCsv.csv
DJI_0036_lr.MP4  DJIFlightRecord_2018-09-30_[10-24-27]-TxtLogToCsv.csv
DJI_0037_lr.MP4  frames


In [36]:
for vid_id in vid_ids:
    
    # Make folder to hold frames
    vid_frames_path = frames_path + vid_id; print(vid_frames_path)
    try: 
        os.mkdir(vid_frames_path)
    except:
        print("failed to mkdir")
        shutil.rmtree(vid_frames_path)
        os.mkdir(vid_frames_path)

    # Grab video, make reader and grab first frame
    vid = group_folder_path +'/'+ vid_id + ".MP4"; print(vid)
    vidcap = cv2.VideoCapture(vid)
    success,image = vidcap.read() # image is np array

    # Grab timestamp info, and first timestamp
    first_ts = vidcap.get(cv2.CAP_PROP_POS_MSEC)
    timestamps = [first_ts]

    count = 0
    success = True
    while success:
        img_path = vid_frames_path +'/frame_'+ format_ix(count) +".jpg";
        cv2.imwrite(img_path, image)     # save frame as JPEG file
        success,image = vidcap.read()
        if success: timestamps.append(vidcap.get(cv2.CAP_PROP_POS_MSEC))
        count += 1
        
    pd_ts = pd.DataFrame({"timestamp":np.round(timestamps, 0).astype(int)}); print(pd_ts.head())
    pd_ts.to_csv(vid_id+"_frame_timestamps.csv", index=False)

../../videos/9.27_lazy_overhead/frames/DJI_0043_lr
../../videos/9.27_lazy_overhead/DJI_0043_lr.MP4
   timestamp
0         33
1         67
2        100
3        133
4        167


In [33]:
# Remove entire folder if necessary

shutil.rmtree("../../videos/9.27_lazy_overhead/frames/DJI_0043_lr")

In [40]:
!ls "../../videos/9.27_lazy_overhead/frames/DJI_0043_lr"

frame_00000.jpg  frame_04945.jpg  frame_09890.jpg  frame_14835.jpg
frame_00001.jpg  frame_04946.jpg  frame_09891.jpg  frame_14836.jpg
frame_00002.jpg  frame_04947.jpg  frame_09892.jpg  frame_14837.jpg
frame_00003.jpg  frame_04948.jpg  frame_09893.jpg  frame_14838.jpg
frame_00004.jpg  frame_04949.jpg  frame_09894.jpg  frame_14839.jpg
frame_00005.jpg  frame_04950.jpg  frame_09895.jpg  frame_14840.jpg
frame_00006.jpg  frame_04951.jpg  frame_09896.jpg  frame_14841.jpg
frame_00007.jpg  frame_04952.jpg  frame_09897.jpg  frame_14842.jpg
frame_00008.jpg  frame_04953.jpg  frame_09898.jpg  frame_14843.jpg
frame_00009.jpg  frame_04954.jpg  frame_09899.jpg  frame_14844.jpg
frame_00010.jpg  frame_04955.jpg  frame_09900.jpg  frame_14845.jpg
frame_00011.jpg  frame_04956.jpg  frame_09901.jpg  frame_14846.jpg
frame_00012.jpg  frame_04957.jpg  frame_09902.jpg  frame_14847.jpg
frame_00013.jpg  frame_04958.jpg  frame_09903.jpg  frame_14848.jpg
frame_00014.jpg  frame_04959.jpg  frame_09904.jpg  frame_14849

In [37]:
#
transform = transforms.Compose([
    transforms.Resize((128)),
    #transforms.RandomCrop(64, pad_if_needed=True),
    transforms.CenterCrop(128),
    transforms.ToTensor(), 
])

# Obnoxious. Images have to be within folder within root
dataset = datasets.ImageFolder(root='./drone', transform=transform)
len(dataset.imgs)

2969

In [39]:
a = torch.randn(1,2); 


tensor([[-2.9886, -0.2219]])

In [0]:
sample = dataset[2968][0].data.cpu()
save_image(sample.data.cpu(), 'sample_image.png')
display(Image('sample_image.png', width=300, unconfined=True))
display(Image('drone/train/frame2968.jpg', width=300, unconfined=True))

In [0]:
# Stitching frames back together into video

img1 = cv2.imread('drone/train/frame100.jpg') # this is a np array
height , width , layers =  img1.shape
fps = 30

video = cv2.VideoWriter('drone.avi', cv2.VideoWriter_fourcc(*"MJPG"), fps, (width,height))

for i in range(count-1):
    img = cv2.imread('drone/train/frame'+str(i)+'.jpg')
    video.write(img)
    
cv2.destroyAllWindows()
video.release()

In [19]:
img1.shape

(720, 1280, 3)

In [0]:
from google.colab import files

files.download('drone.avi')  # from colab to browser download

In [0]:
# Stitching frames back together into video

img1 = cv2.imread('drone/train/frame100.jpg')
img2 = cv2.imread('drone/train/frame101.jpg')
img3 = cv2.imread('drone/train/frame102.jpg')

height , width , layers =  img1.shape

video = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*"MJPG"),1,(width,height))

video.write(img1)
video.write(img2)
video.write(img3)

cv2.destroyAllWindows()
video.release()

writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30,(640,480))
for frame in range(10):
    writer.write(np.random.randint(0, 255, (480,640,3)).astype('uint8'))
writer.release()

In [4]:
# Dowload youtube-dl

!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     3    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     3    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   599    0   599    0     0    446      0 --:--:--  0:00:01 --:--:--   446
100 1655k  100 1655k    0     0   477k      0  0:00:03  0:00:03 --:--:-- 1027k


In [5]:
# download clip
!youtube-dl https://www.youtube.com/watch?v=bR4Gq9qfpnM

[youtube] bR4Gq9qfpnM: Downloading webpage
[youtube] bR4Gq9qfpnM: Downloading video info webpage
[download] Destination: Drone racing - First Person View (FPV)-bR4Gq9qfpnM.mp4
[download] 100% of 24.48MiB in 00:0079MiB/s ETA 00:00known ETA
